In [18]:
import gensim
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [11]:
data_cleaned = pd.read_csv("data/data_cleaned_100k.csv").fillna("")
data_cleaned

,Categorie1,Description
0,INFORMATIQUE,batter acer aspir one ion mah noir compatibl b...
1,TELEPHONIE - GPS,coqu rigid bleu lagon alcatel motif drapeau li...
2,TELEPHONIE - GPS,facad coqu cellular lin shckgal minip marqu ag...
3,TELEPHONIE - GPS,coqu meteor tpu nexus
4,TELEPHONIE - GPS,coqu soupl transparent flex motif keep calm pl...
...,...,...
99995,DECO - LINGE - LUMINAIRE,souvenir franc tour eiffel miniatur prestig co...
99996,LIBRAIRIE,fast track wast fre manufacturing john dav fas...
99997,TELEPHONIE - GPS,fitbag bong giraf houss pochet telephon portab...
99998,DECO - LINGE - LUMINAIRE,grand tableau minn


In [12]:
train_array_token = [line.split(" ") for line in data_cleaned.Description.values]
print(train_array_token[:5])

[['batter', 'acer', 'aspir', 'one', 'ion', 'mah', 'noir', 'compatibl', 'batter', ''], ['coqu', 'rigid', 'bleu', 'lagon', 'alcatel', 'motif', 'drapeau', 'liberi', 'film', 'coqu', 'rigid', 'ultra', 'fin', 'bleu', 'lagon', 'original', 'muzzano', 'motif', 'drapeau', 'liberi', 'alcatel', ''], ['facad', 'coqu', 'cellular', 'lin', 'shckgal', 'minip', 'marqu', 'agree', 'samsungmobil', 'compatibl', 'galaxy', 'minimatier', 'caoutchouc', 'soupl', ''], ['coqu', 'meteor', 'tpu', 'nexus', ''], ['coqu', 'soupl', 'transparent', 'flex', 'motif', 'keep', 'calm', 'play', 'football', 'coqu', 'soupl', 'ultra', 'fin', 'transparent', 'original', 'muzzano', 'motif', 'keep', 'calm', 'play', 'footbal', '']]


In [13]:
Features_dimension = 300
hs = 0
negative = 10

In [14]:
sg = 1
print("Start learning skip-gram Word2Vec")
ts = time.time()
model_sg = gensim.models.Word2Vec(train_array_token, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
te = time.time()
t_learning = te-ts
print("Learning time : %.2f seconds Word2Vec" %t_learning)


sg = 0
print("Start learning CBOW Word2Vec")
ts = time.time()
model_cbow = gensim.models.Word2Vec(train_array_token, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
te = time.time()
t_learning = te-ts
print("Learning time : %.2f seconds Word2Vec" %t_learning)

Start learning skip-gram Word2Vec
Learning time : 72.21 seconds Word2Vec
Start learning CBOW Word2Vec
Learning time : 29.47 seconds Word2Vec


In [20]:
label = data_cleaned["Categorie1"].values
X_train, X_test, y_train, y_test = train_test_split(model_sg, test_size=0.1, random_state=42)

TypeError: Expected sequence or array-like, got <class 'gensim.models.word2vec.Word2Vec'>

In [ ]:
ts = time.time()
lr = LogisticRegression()
lr.fit(X_train, y_train)
te = time.time()
print(te-ts, "sec")

In [6]:
model_sg.most_similar(["montr"])

<ipython-input-6-919232da4ac8>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model_sg.most_similar(["montr"])


[('quartz', 0.8094842433929443),
 ('cadran', 0.7440237998962402),
 ('seiko', 0.7422492504119873),
 ('chronograph', 0.7295635938644409),
 ('analog', 0.7236934900283813),
 ('chrono', 0.7124642729759216),
 ('pulsar', 0.6960623264312744),
 ('mouv', 0.6925901174545288),
 ('caracteris', 0.6881685853004456),
 ('regard', 0.6838310956954956)]

In [7]:
model_cbow.most_similar(["montr"])

<ipython-input-7-479dfe75750b>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model_cbow.most_similar(["montr"])


[('chronograph', 0.8576605319976807),
 ('quartz', 0.8038570284843445),
 ('watch', 0.8025791645050049),
 ('speedway', 0.7981700897216797),
 ('analog', 0.785297155380249),
 ('swiss', 0.7819697856903076),
 ('dial', 0.7502950429916382),
 ('fossil', 0.7464872598648071),
 ('subaqu', 0.7416518330574036),
 ('chrono', 0.7410250306129456)]

In [8]:
model_sg["montr"].shape

<ipython-input-8-952d2a2b1789>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  model_sg["montr"].shape


(300,)

In [17]:
#faire tourner sg sur le max de données avec données nettoyées
#puis prediction avec la regression à comparer le mm modele avec les features de count vectorizer

# Extra Work

pre-trained network


Github du projet[https://github.com/Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors) appris sur 1Giga d'articles de wikipedia en mode **Skip-Gram*

Vous pouvez télécharger ce modèle en suivant ce [lien](https://drive.google.com/file/d/0B0ZXk88koS2KM0pVTktxdG15TkE/view). Dezipez-le puis téléchargez le modèle en indiquant la direction du fichier "fr/bin"

In [ ]:
model_online_dir = "data/fr/fr.bin"
#model_online_dir = "ACOMPLETER/fr.bin"
model_online = gensim.models.Word2Vec.load(model_online_dir)